In [1]:
# Inspiration :D
# https://gitlab.com/ryanlogsdon/pubsub-for-python/-/blob/main/subscriber.py
# https://www.youtube.com/watch?v=xOtrCmPjal8&ab_channel=D-I-Ry
import os
from google.cloud import pubsub_v1
import sys
import time
import pandas as pd
import http.client
import numpy as np
import datetime
from io import StringIO
import functools as ft

In [2]:
! python --version

Python 3.7.4


In [4]:
# Parameters section

location = "Hamburg"

## Create connection to PubSub

In [5]:
credentials_path = 'weather-based-forecasting-v2-c4bde37656a7.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

In [6]:
publisher_obs = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

publisher_forecast = pubsub_v1.PublisherClient(
    publisher_options = pubsub_v1.types.PublisherOptions(
        enable_message_ordering=True,
    )
)

topic_path_obs = 'projects/weather-based-forecasting-v2/topics/observations_topic'
topic_path_forecast = 'projects/weather-based-forecasting-v2/topics/forecasts_topic'

## Download the data

In [7]:
time.sleep(5)



conn = http.client.HTTPSConnection("visual-crossing-weather.p.rapidapi.com")

headers = {
    "X-RapidAPI-Key": "c7b48b7c4emsh8a3b688e71b9896p1be4d0jsnda2e5b27b486",
    "X-RapidAPI-Host": "visual-crossing-weather.p.rapidapi.com"
}


conn.request("GET", "/forecast?location={}&aggregateHours=1&shortColumnNames=0&unitGroup=metric&contentType=csv".format(location),
             headers=headers)

res = conn.getresponse()
data = res.read()
s = str(data, 'utf-8')
data_string = StringIO(s)
df = pd.read_csv(data_string)

In [8]:
df2 = df.copy()
df2.index = pd.to_datetime(df2["Date time"])
df2 = df2.asfreq(freq="1min")
df2 = df2.interpolate(method ='linear', limit_direction ='forward')

df2["Date time"] = df2.index

df3 = df2.iloc[:60*24*10, :]

In [9]:
df3

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2023-01-04 12:00:00,Hamburg,2023-01-04 12:00:00,53.5537,9.99183,"Hamburg, Deutschland",Hamburg,230.700000,10.300000,21.200000,98.000,NaN,100.0,0.300000,1010.000000,0.0,0.0,86.800000,50.800000,NaN,"Rain, Overcast"
2023-01-04 12:01:00,NaN,2023-01-04 12:01:00,53.5537,9.99183,NaN,NaN,230.685000,10.300000,21.206667,98.025,NaN,100.0,0.313333,1009.966667,0.0,0.0,86.830000,50.745000,NaN,NaN
2023-01-04 12:02:00,NaN,2023-01-04 12:02:00,53.5537,9.99183,NaN,NaN,230.670000,10.300000,21.213333,98.050,NaN,100.0,0.326667,1009.933333,0.0,0.0,86.860000,50.690000,NaN,NaN
2023-01-04 12:03:00,NaN,2023-01-04 12:03:00,53.5537,9.99183,NaN,NaN,230.655000,10.300000,21.220000,98.075,NaN,100.0,0.340000,1009.900000,0.0,0.0,86.890000,50.635000,NaN,NaN
2023-01-04 12:04:00,NaN,2023-01-04 12:04:00,53.5537,9.99183,NaN,NaN,230.640000,10.300000,21.226667,98.100,NaN,100.0,0.353333,1009.866667,0.0,0.0,86.920000,50.580000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-14 11:55:00,NaN,2023-01-14 11:55:00,53.5537,9.99183,NaN,NaN,185.816667,6.083333,15.833333,100.000,NaN,19.0,0.000000,1017.333333,0.0,0.0,87.866667,36.141667,2.975,NaN
2023-01-14 11:56:00,NaN,2023-01-14 11:56:00,53.5537,9.99183,NaN,NaN,185.653333,6.086667,15.826667,100.000,NaN,19.0,0.000000,1017.326667,0.0,0.0,87.853333,36.133333,2.980,NaN
2023-01-14 11:57:00,NaN,2023-01-14 11:57:00,53.5537,9.99183,NaN,NaN,185.490000,6.090000,15.820000,100.000,NaN,19.0,0.000000,1017.320000,0.0,0.0,87.840000,36.125000,2.985,NaN


In [10]:
now = datetime.datetime.now()
minutes = now.minute
df3_hour = df3.iloc[minutes-1:minutes+59, :]
df3_hour.head(5)

,Address,Date time,Latitude,Longitude,Resolved Address,Name,Wind Direction,Temperature,Wind Speed,Cloud Cover,Heat Index,Chance Precipitation (%),Precipitation,Sea Level Pressure,Snow Depth,Snow,Relative Humidity,Wind Gust,Wind Chill,Conditions
Date time,,,,,,,,,,,,,,,,,,,,
2023-01-04 12:20:00,NaN,2023-01-04 12:20:00,53.5537,9.99183,NaN,NaN,230.400,10.3,21.333333,98.500,NaN,100.0,0.566667,1009.333333,0.0,0.0,87.40,49.700,NaN,NaN
2023-01-04 12:21:00,NaN,2023-01-04 12:21:00,53.5537,9.99183,NaN,NaN,230.385,10.3,21.340000,98.525,NaN,100.0,0.580000,1009.300000,0.0,0.0,87.43,49.645,NaN,NaN
2023-01-04 12:22:00,NaN,2023-01-04 12:22:00,53.5537,9.99183,NaN,NaN,230.370,10.3,21.346667,98.550,NaN,100.0,0.593333,1009.266667,0.0,0.0,87.46,49.590,NaN,NaN
2023-01-04 12:23:00,NaN,2023-01-04 12:23:00,53.5537,9.99183,NaN,NaN,230.355,10.3,21.353333,98.575,NaN,100.0,0.606667,1009.233333,0.0,0.0,87.49,49.535,NaN,NaN
2023-01-04 12:24:00,NaN,2023-01-04 12:24:00,53.5537,9.99183,NaN,NaN,230.340,10.3,21.360000,98.600,NaN,100.0,0.620000,1009.200000,0.0,0.0,87.52,49.480,NaN,NaN


## Send observations in intervals to PubSub - check Weather_PubSub_check for results

In [1]:
for i in range(3):
    obs = str(df3_hour.iloc[i,:].astype(str).to_dict())
    obs = obs.replace("'", '"')
    index = df3_hour.iloc[i,:].name
    index = str(int(index.value / 10**9)).encode("utf-8")
    obs = obs.encode("utf-8")
    future = publisher_obs.publish(topic_path_obs, obs, ordering_key=index)
    print(f'published message id {future.result()}')
    time.sleep(60)

NameError: name 'df3_hour' is not defined